In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [4]:
urlpage = "http://sysbio.gzzoc.com/rpfdb/browse.html#@Scerevisiae@GSE100626"
driver = webdriver.Firefox()
try:
    driver.get(urlpage)
except TimeoutException:
    time.sleep(5)
    driver.get(urlpage)

dic = {}
tabs = driver.find_elements_by_xpath("//ul[@id = 'ScerevisiaeStudyNav']/li[contains(@class, 'tab')]/a")

In [5]:
for tab in tabs:
    link = tab.get_attribute("href")
    ActionChains(driver).move_to_element(tab).send_keys(Keys.ENTER).perform()
    tab.click()
    time.sleep(3)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "ScerevisiaeDetail")))
    # WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located)
    name = tab.text
    if name != driver.find_element_by_xpath(
        "/html/body/div[2]/div/div[2]/div[1]/div[22]/div/div[@id='Scerevisiae_{}_Detail']/table/tbody/tr[2]/td[2]/a".format(name)).text:
        tab.click()
        time.sleep(3)

    dic[name] = {}
    dic[name]["pmid"] = driver.find_element_by_xpath(
        "/html/body/div[2]/div/div[2]/div[1]/div[22]/div/div[@id='Scerevisiae_{}_Detail']/table/tbody/tr[2]/td[4]/a".format(name)).get_attribute(
        "href")
    dic[name]["accession"] = driver.find_element_by_xpath(
        "/html/body/div[2]/div/div[2]/div[1]/div[22]/div/div[@id='Scerevisiae_{}_Detail']/table/tbody/tr[2]/td[2]/a".format(name)).get_attribute(
        "href")
    dic[name]["title"] = driver.find_element_by_xpath("//div[@id='Scerevisiae_{}_Detail']/table/tbody/tr[1]/td[@colspan ='3']/a".format(name)).text
    # dic[name]["strain"] = driver.find_element_by_xpath("/html/body/div[2]/div/div[2]/div[1]/div[22]/div/div[@class='tab-pane active']/table/tbody/tr[3]/td[2]").text.split(":")[1]
    strain1 = driver.find_element_by_xpath(
        "/html/body/div[2]/div/div[2]/div[1]/div[22]/div/div[@id='Scerevisiae_{}_Detail']/table/tbody/tr[3]/td[2]".format(name).format(name)).text
    strain2 = strain1.split(":")
    try:
        strain3 = strain2[1]
    except IndexError:
        strain3 = strain2[0]
    dic[name]["strain"] = strain3



In [22]:
driver = webdriver.Firefox()
for GSE in dic:
    if "XXX" not in GSE:
        acc_url = dic[GSE]["accession"]
        # pmid = dic[GSE]["pmid"]
        # strain = dic[GSE]["strain"]
        try:
            driver.get(acc_url)
        except TimeoutException:
            time.sleep(5)
            driver.get(acc_url)
        if driver.find_element_by_tag_name("body").get_attribute("id") == "ui-ncbiexternallink-5":
            if len(driver.find_elements_by_xpath("/html/body/div[1]/div[1]/form/div[1]/div[5]/div/div[3]/div/h2[text() = 'Search results']"))>0:
                try:
                    acc_url = driver.find_element_by_xpath(
                        "//div[@class='rprt']/div[2]/p/a[text()='{}']".format(dic[GSE]["title"])).get_attribute("href")
                except NoSuchElementException:
                    try:
                        acc_url = driver.find_element_by_xpath("//div[@class='rprt']/div[2]/div[@class = 'supp']/p[@class = 'desc'][text()='{}']//ancestor::div[@class = 'rslt']//child::p/a".format(dic[GSE]["title"])).get_attribute("href")
                    except NoSuchElementException:
                        print("To look" + GSE)
                        continue
                try:
                    driver.get(acc_url)
                except TimeoutException:
                    time.sleep(5)
                    driver.get(acc_url)
            else:
                pass

        elif driver.find_element_by_tag_name("body").get_attribute("id") == "ui-ncbiexternallink-4":
            dic[GSE]["links"] = []
            print("No links found for" + GSE)
            continue
        else:
            print("Hmm "+GSE + driver.find_element_by_tag_name("body").get_attribute("id"))
        if acc_url == "Not found":
            dic[GSE]["pubmed"] = "Not found"
            dic[GSE]["desc"] = "Not found"
            dic[GSE]["desc2"] = "Not found"
        else:
            try:
                dic[GSE]["pubmed"] = driver.find_element_by_id("PubMed").get_attribute("href")
            except NoSuchElementException:
                dic[GSE]["pubmed"] = "not found"
            try:
                dic[GSE]["desc"] = driver.find_element_by_css_selector(".Title > h3:nth-child(2)").text
            except NoSuchElementException:
                dic[GSE]["desc"] = driver.find_element_by_css_selector(".Title > h2:nth-child(1)").text
            results = driver.find_element_by_id("SRA Experiments").get_attribute("href")
            driver.get(results)
            try:
                driver.find_element_by_css_selector(
                    "div.results_settings:nth-child(1) > ul:nth-child(1) > li:nth-child(2) > a:nth-child(1)").click()
                WebDriverWait(driver, 100000).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[type='radio'][value='200']")))
                driver.find_element_by_css_selector("input[type='radio'][value='200']").click()
            except NoSuchElementException :
                print("One page ? " + results)
                # nr_pages = driver.find_element_by_id("pageno").get_attribute("last")
                # print(nr_pages)
            next_page = driver.find_elements_by_css_selector("div.pagination:nth-child(1) > a:nth-child(4)")
            dic[GSE]["links"] = []
            # try:
            if driver.find_elements_by_xpath("h2[text()='Links from BioProject']"):
                dic[GSE]["links"] = [results]
            else:
                dic[GSE]["links"] = dic[GSE]["links"] + [x.get_attribute("href") for x in driver.find_elements_by_xpath("//*[contains(@class, 'title')]/a")]
                while (len(next_page) > 0):
                    next_page[0].click()
                    dic[GSE]["links"] = dic[GSE]["links"] + [x.get_attribute("href") for x in
                                     driver.find_elements_by_xpath("//*[contains(@class, 'title')]/a")]
                    next_page = driver.find_elements_by_css_selector("div.pagination:nth-child(1) > a:nth-child(4)")
            # except NoSuchElementException:


Hmm GSE100626ui-ncbiexternallink-7
Hmm GSE13750ui-ncbiexternallink-7
Hmm GSE34082ui-ncbiexternallink-7
Hmm GSE34438ui-ncbiexternallink-7
One page ? https://www.ncbi.nlm.nih.gov/sra?linkname=bioproject_sra_all&from_uid=156379
Hmm GSE41590ui-ncbiexternallink-7
Hmm GSE45366ui-ncbiexternallink-7
No links found forGSE50049
Hmm GSE51164ui-ncbiexternallink-7
One page ? https://www.ncbi.nlm.nih.gov/sra?linkname=bioproject_sra_all&from_uid=221377
Hmm GSE51532ui-ncbiexternallink-7
No links found forGSE52119
Hmm GSE52968ui-ncbiexternallink-7
One page ? https://www.ncbi.nlm.nih.gov/sra?linkname=bioproject_sra_all&from_uid=231673
Hmm GSE55400ui-ncbiexternallink-7
Hmm GSE56622ui-ncbiexternallink-7
Hmm GSE58321ui-ncbiexternallink-7
Hmm GSE59573ui-ncbiexternallink-7
Hmm GSE61011ui-ncbiexternallink-7
One page ? https://www.ncbi.nlm.nih.gov/sra?linkname=bioproject_sra_all&from_uid=260072
Hmm GSE61753ui-ncbiexternallink-7
Hmm GSE63789ui-ncbiexternallink-7
Hmm GSE64304ui-ncbiexternallink-7
One page ? http

In [25]:
# driver = webdriver.Firefox()
with open("SRA_parser2.tab","w") as parser:
    parser.write("Accession\tAccession link\tPMID\tStrain\tPubmed\tdesc\tdesc2\tSRA experiments link\tSRA\n")
    for GSE in dic:
        if "XXX" not in GSE:
            for sra_link in dic[GSE]["links"]:
                if not sra_link:
                    continue
                try:
                    driver.get(sra_link)
                except (TimeoutException, WebDriverException) as e:
                    time.sleep(5)
                    driver.get(sra_link)
                sra = [x.text for x in
                       driver.find_elements_by_xpath("//a[contains(@href, 'trace.ncbi.nlm.nih.gov/Traces/sra/?run=')]")]
                # try:
                desc2 = driver.find_element_by_xpath("//div[text()='Sample: ']/span").text.split("\n")[0]
                if 'All runs' in desc2:
                    desc2 = driver.find_element_by_xpath("/html/body/div[1]/div[1]/form/div[1]/div[5]/div/div[5]/p/b").text
                if 'Generic sample from Saccharomyces cerevisiae' in desc2:
                    desc2 = driver.find_element_by_xpath("//div[text()='Name: ']/span").text
                # except IndexError:
                #     desc2 = "to look"
                for i in sra:
                    parser.write(
                        "{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(GSE,dic[GSE]["accession"],dic[GSE]["pmid"],
                                                                      dic[GSE]["strain"] ,dic[GSE]["pubmed"],dic[GSE]["desc"],
                                                                      desc2,sra_link,i))

In [38]:
import pandas as pd
all_data = pd.read_table("SRA_parser.tab", sep = "\t")
print(len(all_data.index))
RNA_seq = all_data[all_data["desc2"].str.contains("RNA|rna|transcriptome|RiboMeth|TP")]
less_RNA_seq = all_data[~all_data["desc2"].str.contains("RNA|rna|transcriptome|RiboMeth|TP")]
print(len(less_RNA_seq.index))
just_ribo = less_RNA_seq[less_RNA_seq["desc2"].str.contains('ribo|ootprint|Ribo|traductome|FP|RPF')]
print(len(just_ribo.index))
maybe_ribo = less_RNA_seq[~less_RNA_seq["desc2"].str.contains('ribo|ootprint|Ribo|traductome|FP|RPF')]
print(len(maybe_ribo.index))
contains_maybe_rna = []
contains_RNA = []
normally_just_ribo = ["GSE58321","GSE59573","GSE61011","GSE61012","GSE79622"]
for GSE in dic:
    if len(RNA_seq[RNA_seq["Accession"]==GSE].index)!=0:
        contains_RNA.append(GSE)
    if len(just_ribo[just_ribo["Accession"]==GSE].index)==0:
        #GSE50049, GSE52119 is empty
        #-GSE58321, -GSE59573, -GSE61011,-GSE61012,PRJNA231536 to look if RNA-seq
        contains_maybe_rna.append(GSE)
        # print(GSE)
        # if GSE not in contains_RNA:
            # print("contains no RNA " + GSE)
filtered_data = all_data[(all_data["desc2"].str.contains('ribo|ootprint|Ribo|traductome|FP|RP|RNAse'))|
                         ((all_data["Accession"].isin (contains_RNA)&(~all_data["desc2"].str.contains("RNA|rna|transcriptome|RiboMeth|TP|Polysome-seq"))))]
print(len(filtered_data.index))
for GSE in dic:
    if len(filtered_data[filtered_data["Accession"]==GSE].index)==0:
        print(GSE)

1216
807
523
284
691
GSE50049
GSE52119
GSE58321
GSE59573
GSE61011
GSE61012
GSE64304
GSE79622
GSE85686
GSE89704
GSE90919
GSE98333
PRJNA231536
